# Sale Prediction Project
## Problem Statement:
Nowadays, shopping malls and Big Marts keep track of individual item sales data in
order to forecast future client demand and adjust inventory management. In a data
warehouse, these data stores hold a significant amount of consumer information and
particular item details. By mining the data store from the data warehouse, more
anomalies and common patterns can be discovered.

### Approach: 
The classical machine learning tasks like Data Exploration, Data Cleaning,
Feature Engineering, Model Building and Model Testing. Try out different machine
learning algorithms that’s best fit for the above case.

### Results: 
You have to build a solution that should able to predict the sales of the
different stores of Big Mart according to the provided dataset.

### Datasrt link: https://www.kaggle.com/datasets/brijbhushannanda1979/bigmart-sales-data

## Dataset Background:
We have train (8523) and test (5681) data set, train data set has both input and output
variable(s). We need to predict the sales for test data set.

**Item_Identifier:** Unique product ID 2

**Item_Weight:** Weight of product

**Item_Fat_Content:** Whether the product is low fat or not

**Item_Visibility:** The % of total display area of all products in a store allocated to the
particular product

**Item_Type:** The category to which the product belongs

**Item_MRP:** Maximum Retail Price (list price) of the product

**Outlet_Identifier:** Unique store ID

**Outlet_Establishment_Year:** The year in which store was established

**Outlet_Size:** The size of the store in terms of ground area covered

**Outlet_Location_Type:** The type of city in which the store is located

**Outlet_Type:** Whether the outlet is just a grocery store or some sort of supermarket

**Item_Outlet_Sales:** Sales of the product in the particulat store. This is the outcome
variable to be predicted.


In [5]:
#importing basics libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# Modelling-
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split,GridSearchCV, RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import pickle

In [2]:
df_train = pd.read_csv('data/Train.csv')
df_test = pd.read_csv('data/Test.csv')

In [4]:
df_train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
df_train.shape

(8523, 12)

In [5]:
df_train.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

In [6]:
df_test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [7]:
df_test.shape

(5681, 11)

In [8]:
df_train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [10]:
df_train.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [11]:
df_test.isnull().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [12]:
df_train.duplicated().sum()

0

In [13]:
df_train.head(3)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700


In [14]:
df_train['Item_Identifier'].value_counts()

Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: count, Length: 1559, dtype: int64

In [15]:
df_train['Item_Identifier'].value_counts()

Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: count, Length: 1559, dtype: int64

In [16]:
df_train['Outlet_Establishment_Year'].unique()

array([1999, 2009, 1998, 1987, 1985, 2002, 2007, 1997, 2004], dtype=int64)

In [17]:
# define numerical & categorical columns in train data
numeric_features = [feature for feature in df_train.columns if df_train[feature].dtype != 'O']
categorical_features = [feature for feature in df_train.columns if df_train[feature].dtype == 'O']

# print numerical & categorical columns in train data
print('We have {} numerical features in train data and they as as follows : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features train data and they as as follows: {}'.format(len(categorical_features), categorical_features))

We have 5 numerical features in train data and they as as follows : ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year', 'Item_Outlet_Sales']

We have 7 categorical features train data and they as as follows: ['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']


In [18]:
print('Number of unique data points in categorical features in Train data')
print('Number of unique data points in Item_Identifier:', df_train['Item_Identifier'].unique())
print('Number of unique data points in Item_Fat_Content:', df_train['Item_Fat_Content'].unique())
print('Number of unique data points in Item_Type:',df_train['Item_Type'].unique())
print('Number of unique data points in Outlet_Identifier:', df_train['Outlet_Identifier'].unique())
print('Number of unique data points in Outlet_Size:', df_train['Outlet_Size'].unique())
print('Number of unique data points in Outlet_Location_Type:', df_train['Outlet_Location_Type'].unique())
print('Number of unique data points in Outlet_Type:', df_train['Outlet_Type'].unique())

Number of unique data points in categorical features in Train data
Number of unique data points in Item_Identifier: ['FDA15' 'DRC01' 'FDN15' ... 'NCF55' 'NCW30' 'NCW05']
Number of unique data points in Item_Fat_Content: ['Low Fat' 'Regular' 'low fat' 'LF' 'reg']
Number of unique data points in Item_Type: ['Dairy' 'Soft Drinks' 'Meat' 'Fruits and Vegetables' 'Household'
 'Baking Goods' 'Snack Foods' 'Frozen Foods' 'Breakfast'
 'Health and Hygiene' 'Hard Drinks' 'Canned' 'Breads' 'Starchy Foods'
 'Others' 'Seafood']
Number of unique data points in Outlet_Identifier: ['OUT049' 'OUT018' 'OUT010' 'OUT013' 'OUT027' 'OUT045' 'OUT017' 'OUT046'
 'OUT035' 'OUT019']
Number of unique data points in Outlet_Size: ['Medium' nan 'High' 'Small']
Number of unique data points in Outlet_Location_Type: ['Tier 1' 'Tier 3' 'Tier 2']
Number of unique data points in Outlet_Type: ['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3']


## Data Preprocessing 
1. Drop null values 
2. Drop redundant features 
3. Feature encoding 

In [19]:
# Drop rows with missing values
df_train = df_train.dropna()

#creating our new column for both datasets
df_train['Outlet_Age']= df_train ['Outlet_Establishment_Year'].apply(lambda year: 2023 - year)

# Standardize values in the 'Item_Fat_Content' column
df_train['Item_Fat_Content'] = df_train['Item_Fat_Content'].replace({'LF': 'Low Fat', 'low fat': 'Low Fat', 'reg': 'Regular'})
df_train.drop(['Item_Identifier','Outlet_Identifier', 'Item_Visibility', 'Outlet_Establishment_Year'], axis = 1, inplace  = True)

In [20]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4650 entries, 0 to 8522
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Item_Weight           4650 non-null   float64
 1   Item_Fat_Content      4650 non-null   object 
 2   Item_Type             4650 non-null   object 
 3   Item_MRP              4650 non-null   float64
 4   Outlet_Size           4650 non-null   object 
 5   Outlet_Location_Type  4650 non-null   object 
 6   Outlet_Type           4650 non-null   object 
 7   Item_Outlet_Sales     4650 non-null   float64
 8   Outlet_Age            4650 non-null   int64  
dtypes: float64(3), int64(1), object(5)
memory usage: 363.3+ KB


In [21]:
df_train['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [22]:
df_train.head()

,Item_Weight,Item_Fat_Content,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Age
0,9.300,Low Fat,Dairy,249.8092,Medium,Tier 1,Supermarket Type1,3735.1380,24
1,5.920,Regular,Soft Drinks,48.2692,Medium,Tier 3,Supermarket Type2,443.4228,14
2,17.500,Low Fat,Meat,141.6180,Medium,Tier 1,Supermarket Type1,2097.2700,24
4,8.930,Low Fat,Household,53.8614,High,Tier 3,Supermarket Type1,994.7052,36
5,10.395,Regular,Baking Goods,51.4008,Medium,Tier 3,Supermarket Type2,556.6088,14


In [25]:
df_train['Item_Outlet_Sales'].max()

10256.649

In [26]:
df_train['Item_Outlet_Sales'].min()

69.2432

In [28]:
df_train.shape

(4650, 9)

In [29]:
df_train['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    3004
Regular    1646
Name: count, dtype: int64

In [30]:
df_train['Outlet_Size'].value_counts()

Outlet_Size
Small     1860
Medium    1858
High       932
Name: count, dtype: int64

In [31]:
# define numerical & categorical columns in train data
numeric_features = [feature for feature in df_train.columns if df_train[feature].dtype != 'O']
categorical_features = [feature for feature in df_train.columns if df_train[feature].dtype == 'O']

# print numerical & categorical columns in train data
print('We have {} numerical features in train data and they as as follows : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features train data and they as as follows: {}'.format(len(categorical_features), categorical_features))

We have 4 numerical features in train data and they as as follows : ['Item_Weight', 'Item_MRP', 'Item_Outlet_Sales', 'Outlet_Age']

We have 5 categorical features train data and they as as follows: ['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']


## Feature Encoding 
1. Label Encoding 
2. One-Hot-Encoding


Ordinal variables:

Item_Fat_Content
Outlet_Size
Outlet_Location_Type

Nominal variables:

Item_Identifier
Item_Type
Outlet_Identifier
Outlet_Type

In [32]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define the lists of categorical and numerical features
numerical_features = ['Item_Weight', 'Item_MRP', 'Item_Outlet_Sales']
categorical_features = ['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

le = LabelEncoder()
Label = ['Item_Fat_Content','Outlet_Size','Outlet_Location_Type']

for i in Label:
    df_train[i] = le.fit_transform(df_train[i])

In [33]:
df_train.head()

,Item_Weight,Item_Fat_Content,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Age
0,9.300,0,Dairy,249.8092,1,0,Supermarket Type1,3735.1380,24
1,5.920,1,Soft Drinks,48.2692,1,2,Supermarket Type2,443.4228,14
2,17.500,0,Meat,141.6180,1,0,Supermarket Type1,2097.2700,24
4,8.930,0,Household,53.8614,0,2,Supermarket Type1,994.7052,36
5,10.395,1,Baking Goods,51.4008,1,2,Supermarket Type2,556.6088,14


In [34]:
#one hot encoding
cols = ['Item_Type','Outlet_Type']
# Apply one-hot encoder
oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False, drop = 'first')
oh_encoder_df_train = pd.DataFrame(oh_encoder.fit_transform(df_train[cols])).astype('int64')

#get feature columns
oh_encoder_df_train.columns = oh_encoder.get_feature_names_out(cols)


# One-hot encoding removed index; put it back
oh_encoder_df_train.index = df_train.index

# Add one-hot encoded columns to our main df new name: tr_fe, te_fe (means feature engeenired) 
df_train = pd.concat([df_train, oh_encoder_df_train], axis=1)

df_train = df_train.drop(['Item_Type', 'Outlet_Type'], axis = 1)

c:\Users\HP\anaconda3\envs\venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [35]:
df_train.head()

,Item_Weight,Item_Fat_Content,Item_MRP,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,Outlet_Age,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,...,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Type_Supermarket Type2
0,9.300,0,249.8092,1,0,3735.1380,24,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5.920,1,48.2692,1,2,443.4228,14,0,0,0,...,0,0,0,0,0,0,0,1,0,1
2,17.500,0,141.6180,1,0,2097.2700,24,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,8.930,0,53.8614,0,2,994.7052,36,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,10.395,1,51.4008,1,2,556.6088,14,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

col_to_scale = ['Item_Weight', 'Item_MRP', 'Item_Outlet_Sales', 'Outlet_Age']

for col in col_to_scale:
    # Reshape the column to a 2D array with a single column
    col_data = df_train[col].values.reshape(-1, 1)
    
    # Fit and transform the scaler on the reshaped data
    df_train[col] = scaler.fit_transform(col_data)


In [37]:
X = df_train.drop(['Item_Outlet_Sales'], axis = 1)
y = df_train['Item_Outlet_Sales']

In [38]:
# Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [39]:
len(X_train)


3720

In [40]:
len(X_test)

930

In [41]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [42]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.7334
- Mean Absolute Error: 0.5386
- R2 Score: 0.4687
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.6985
- Mean Absolute Error: 0.5254
- R2 Score: 0.4866


Lasso
Model performance for Training set
- Root Mean Squared Error: 1.0062
- Mean Absolute Error: 0.8015
- R2 Score: 0.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.9749
- Mean Absolute Error: 0.7833
- R2 Score: -0.0000


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.7334
- Mean Absolute Error: 0.5386
- R2 Score: 0.4687
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.6985
- Mean Absolute Error: 0.5253
- R2 Score: 0.4867


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 0.6648
- Mean Absolute Error: 0.4942
- R2 Score: 0.5635
----------------------

In [45]:
import pandas as pd

def compare_models(models, X_train, y_train, X_test, y_test):
    results = []
    for model_name, model in models.items():
        model_results = evaluate_model(model, X_train, y_train, X_test, y_test)
        model_results['Model'] = model_name
        results.append(model_results)

    return pd.DataFrame(results)


In [43]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model on the training data
    model.fit(X_train, y_train)

    # Make predictions on the training and test data
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calculate evaluation metrics for training data
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mse_train = mean_squared_error(y_train, y_train_pred)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(y_train, y_train_pred)

    # Calculate evaluation metrics for test data
    mae_test = mean_absolute_error(y_test, y_test_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(y_test, y_test_pred)

    # Calculate cross-validation RMSE
    cross_val_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cross_val_rmse = np.sqrt(-cross_val_scores)

    return {
        'MAE_train': mae_train,
        'MSE_train': mse_train,
        'RMSE_train': rmse_train,
        'R^2_train': r2_train,
        'MAE_test': mae_test,
        'MSE_test': mse_test,
        'RMSE_test': rmse_test,
        'R^2_test': r2_test,
        'Cross_Val_RMSE': cross_val_rmse.mean()
    }


In [46]:
# Use the compare_models function
results_df = compare_models(models, X_train, y_train, X_test, y_test)

# Sort the results by MAE
results_df.sort_values(by='MAE_test', ascending=True, inplace=True)

# Save the results to a CSV file
results_df.to_csv("model_comparison_results.csv", index=False)


In [47]:
result = pd.read_csv('model_comparison_results.csv')

In [48]:
result

,MAE_train,MSE_train,RMSE_train,R^2_train,MAE_test,MSE_test,RMSE_test,R^2_test,Cross_Val_RMSE,Model
0,0.538614,0.537874,0.733399,0.468710,0.525341,0.487895,0.698495,0.486652,0.737300,Ridge
1,0.538642,0.537872,0.733398,0.468711,0.525392,0.487969,0.698548,0.486574,0.737344,Linear Regression
2,0.415775,0.304746,0.552038,0.698985,0.551887,0.542640,0.736641,0.429050,0.791857,CatBoosting Regressor
3,0.563010,0.558299,0.747194,0.448535,0.561199,0.545188,0.738368,0.426370,0.767566,AdaBoost Regressor
4,0.213319,0.086317,0.293798,0.914739,0.572295,0.585760,0.765349,0.383682,0.793609,Random Forest Regressor
5,0.289314,0.151533,0.389273,0.850321,0.586266,0.628974,0.793079,0.338213,0.836614,XGBRegressor
6,0.494157,0.441953,0.664795,0.563457,0.607469,0.640694,0.800434,0.325881,0.817040,K-Neighbors Regressor
7,0.000000,0.000000,0.000000,1.000000,0.775355,1.100684,1.049135,-0.158106,1.084215,Decision Tree
8,0.801487,1.012392,1.006177,0.000000,0.783318,0.950435,0.974902,-0.000018,1.006105,Lasso
